In [9]:
# !pip install stable-baselines3[extra]
# !pip install ipykernel
# !pip install gym

In [1]:
import os
import gymnasium as gym

In [45]:
environment_name = 'CartPole-v0'

In [46]:
env = gym.make(environment_name, render_mode="human")

f:\anaconda\Anaconda3\envs\ReinforcementLearning\Lib\site-packages\gymnasium\envs\registration.py:513: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


In [6]:
episodes = 10
for episode in range(episodes):
    observation, info = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        observation, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        score += reward
    print(f"Episode {episode} ended with score {score}")
env.close()

Episode 0 ended with score 14.0
Episode 1 ended with score 19.0
Episode 2 ended with score 20.0
Episode 3 ended with score 30.0
Episode 4 ended with score 9.0
Episode 5 ended with score 14.0
Episode 6 ended with score 19.0
Episode 7 ended with score 19.0
Episode 8 ended with score 14.0
Episode 9 ended with score 16.0


# Training

In [7]:
# Directory where the training data and model params are saved
log_path = os.path.join("Training", "Logs")
log_path

'Training\\Logs'

In [48]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [111]:
env = gym.make(environment_name, render_mode="human")
env = DummyVecEnv([lambda: env])

f:\anaconda\Anaconda3\envs\ReinforcementLearning\Lib\site-packages\gymnasium\envs\registration.py:513: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


In [56]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [10]:
PPO??

Init signature:
PPO(
    policy: Union[str, Type[stable_baselines3.common.policies.ActorCriticPolicy]],
    env: Union[gymnasium.core.Env, ForwardRef('VecEnv'), str],
    learning_rate: Union[float, Callable[[float], float]] = 0.0003,
    n_steps: int = 2048,
    batch_size: int = 64,
    n_epochs: int = 10,
    gamma: float = 0.99,
    gae_lambda: float = 0.95,
    clip_range: Union[float, Callable[[float], float]] = 0.2,
    clip_range_vf: Union[NoneType, float, Callable[[float], float]] = None,
    normalize_advantage: bool = True,
    ent_coef: float = 0.0,
    vf_coef: float = 0.5,
    max_grad_norm: float = 0.5,
    use_sde: bool = False,
    sde_sample_freq: int = -1,
    rollout_buffer_class: Optional[Type[stable_baselines3.common.buffers.RolloutBuffer]] = None,
    rollout_buffer_kwargs: Optional[Dict[str, Any]] = None,
    target_kl: Optional[float] = None,
    stats_window_size: int = 100,
    tensorboard_log: Optional[str] = None,
    policy_kwargs: Optional[Dict[str, Any]]

In [57]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_6


-----------------------------
| time/              |      |
|    fps             | 45   |
|    iterations      | 1    |
|    time_elapsed    | 44   |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 44           |
|    iterations           | 2            |
|    time_elapsed         | 92           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0074830353 |
|    clip_fraction        | 0.0413       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.688       |
|    explained_variance   | 0.00544      |
|    learning_rate        | 0.0003       |
|    loss                 | 5.62         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00646     |
|    value_loss           | 53.1         |
------------------------------------------
----------------

# Save and Reload model

In [15]:
PPO_PATH = os.path.join("Training", "Saved Models", "PPO_model_CartPole")

In [16]:
model.save(PPO_PATH)

In [17]:
del model

In [19]:
model = PPO.load(PPO_PATH, env=env) 

In [20]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_3


-----------------------------
| time/              |      |
|    fps             | 47   |
|    iterations      | 1    |
|    time_elapsed    | 42   |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 45           |
|    iterations           | 2            |
|    time_elapsed         | 89           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0036031997 |
|    clip_fraction        | 0.0222       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.562       |
|    explained_variance   | 0.898        |
|    learning_rate        | 0.0003       |
|    loss                 | 0.376        |
|    n_updates            | 110          |
|    policy_gradient_loss | -0.00044     |
|    value_loss           | 5.15         |
------------------------------------------
----------------

# Evaluation

In [21]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

f:\anaconda\Anaconda3\envs\ReinforcementLearning\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [22]:
env.close()

In [27]:
env.reset()

(array([ 0.04675544, -0.00645232, -0.02603617,  0.00134899], dtype=float32),
 {})

In [29]:
episodes = 10
for episode in range(episodes):
    observation, info = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(observation=observation) # using model
        observation, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        score += reward
    print(f"Episode {episode} ended with score {score}")
env.close()

Episode 0 ended with score 200.0
Episode 1 ended with score 200.0
Episode 2 ended with score 200.0
Episode 3 ended with score 200.0
Episode 4 ended with score 200.0
Episode 5 ended with score 200.0
Episode 6 ended with score 200.0
Episode 7 ended with score 200.0
Episode 8 ended with score 200.0
Episode 9 ended with score 200.0


In [33]:
model.predict(observation=observation)[0]

array(1, dtype=int64)

In [34]:
# obs = env.reset()
# while True:
#     action, _states = model.predict(obs)
#     obs, rewards, done, info = env.step(action)
#     env.render()
#     if done: 
#         print('info', info)
#         break

# viewing logs in Tensorboard

In [38]:
training_log_path = os.path.join(log_path, "PPO_3")

In [39]:
training_log_path

'Training\\Logs\\PPO_3'

In [ ]:
!tensorboard --logdir={training_log_path}

# Using Callback

In [59]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [60]:
save_path = os.path.join("Training", "Saved Models")

In [61]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path,
                             verbose=1)

In [62]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path) 

Using cpu device


In [63]:
model.learn(total_timesteps=40000, callback=eval_callback)

Logging to Training\Logs\PPO_7


-----------------------------
| time/              |      |
|    fps             | 44   |
|    iterations      | 1    |
|    time_elapsed    | 45   |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 44           |
|    iterations           | 2            |
|    time_elapsed         | 91           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0081946235 |
|    clip_fraction        | 0.101        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.686       |
|    explained_variance   | 0.00103      |
|    learning_rate        | 0.0003       |
|    loss                 | 7.24         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0154      |
|    value_loss           | 52.7         |
------------------------------------------
----------------

f:\anaconda\Anaconda3\envs\ReinforcementLearning\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=200.00 +/- 0.00
Episode length: 200.00 +/- 0.00
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 200          |
|    mean_reward          | 200          |
| time/                   |              |
|    total_timesteps      | 10000        |
| train/                  |              |
|    approx_kl            | 0.0073129227 |
|    clip_fraction        | 0.0599       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.609       |
|    explained_variance   | 0.248        |
|    learning_rate        | 0.0003       |
|    loss                 | 28.2         |
|    n_updates            | 40           |
|    policy_gradient_loss | -0.0144      |
|    value_loss           | 67.9         |
------------------------------------------
New best mean reward!
Stopping training because the mean reward 200.00  is above the threshold 200


In [82]:
env.close()

In [81]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [ ]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
    #     break

In [116]:
env.close()

In [97]:
env.reset()

array([[-0.02872264,  0.0004672 ,  0.03527243, -0.01187526]],
      dtype=float32)

In [93]:
action, _ = model.predict(observation=observation)
action

array([1], dtype=int64)

In [99]:
env.step(action)

(array([[ 0.0048127 ,  0.4099528 , -0.01049039, -0.6201391 ]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{'TimeLimit.truncated': False}])

In [113]:
episodes = 100
for episode in range(episodes):
    observation = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(observation=observation) # using model
        observation, reward, terminated, truncated = env.step(action)
        done = terminated or truncated
        score += reward
    print(f"Episode {episode} ended with score {score}")


Episode 0 ended with score [1.]
Episode 1 ended with score [1.]
Episode 2 ended with score [1.]
Episode 3 ended with score [1.]
Episode 4 ended with score [1.]
Episode 5 ended with score [1.]
Episode 6 ended with score [1.]
Episode 7 ended with score [1.]
Episode 8 ended with score [1.]
Episode 9 ended with score [1.]
Episode 10 ended with score [1.]
Episode 11 ended with score [1.]
Episode 12 ended with score [1.]
Episode 13 ended with score [1.]
Episode 14 ended with score [1.]
Episode 15 ended with score [1.]
Episode 16 ended with score [1.]
Episode 17 ended with score [1.]
Episode 18 ended with score [1.]
Episode 19 ended with score [1.]
Episode 20 ended with score [1.]
Episode 21 ended with score [1.]
Episode 22 ended with score [1.]
Episode 23 ended with score [1.]
Episode 24 ended with score [1.]
Episode 25 ended with score [1.]
Episode 26 ended with score [1.]
Episode 27 ended with score [1.]
Episode 28 ended with score [1.]
Episode 29 ended with score [1.]
Episode 30 ended wit

# Change Policies
## Change neural network arcitecture

In [58]:
net_arch = [dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]
net_arch

[{'pi': [128, 128, 128, 128], 'vf': [128, 128, 128, 128]}]

In [ ]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path, policy_kwargs=dict(net_arch=net_arch))

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

# Using Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN
# and do with the same way before